<a href="https://colab.research.google.com/github/bahramzada/ramizrovshen-gpt2-poetry/blob/main/Ramiz_Rovshan_Poetry_gpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datasets import Dataset

def load_poems(path):
    with open(path, "r", encoding="utf-8") as f:
        poems = f.read().split('\n\n')  # hər şeiri ayırır
    return [{"text": poem.strip()} for poem in poems if poem.strip()]

dataset = Dataset.from_list(load_poems("cleaned_poems.txt"))
dataset.save_to_disk("poems_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/326 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_from_disk
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling

dataset = load_from_disk("poems_dataset")
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling, AutoTokenizer, AutoModelForCausalLM

# Tokenizer və model
tokenizer.pad_token = tokenizer.eos_token

def tokenize_fn(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = dataset.map(
    tokenize_fn,
    batched=True,
    remove_columns=dataset.column_names
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

training_args = TrainingArguments(
    output_dir="./gpt2-poetry-ramizrovshen",
    per_device_train_batch_size=2,
    num_train_epochs=100,
    save_steps=500,
    logging_steps=100,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

trainer.train()

Map:   0%|          | 0/326 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: bahramzada (bahramzada-unec-business-school) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
100,3.316700
200,2.841200
300,2.648200
400,2.445800
500,2.354100
600,2.231200
700,2.110000
800,2.063800
900,1.932600
1000,1.900300


TrainOutput(global_step=16300, training_loss=0.37875895906811113, metrics={'train_runtime': 3650.3866, 'train_samples_per_second': 8.931, 'train_steps_per_second': 4.465, 'total_flos': 2129530060800000.0, 'train_loss': 0.37875895906811113, 'epoch': 100.0})

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

model_path = "/content/gpt2-poetry-ramizrovshen/checkpoint-16000"  # Son checkpoint

tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_path)

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
prompt = "Qadınım"
result = generator(prompt, max_length=35, num_return_sequences=1)
print(result[0]['generated_text'])

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=35) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Qadınım var göyüm var
Quşam qəfəsim yoxdu
Uçmağa hər şeyim var
Təkcə həvəsim yoxdu
Yol getməyə yolum var
Qulluğum var qulum var
Xərcləməyə pulum var
Təkcə həvəsim yoxdu
Nə dərd var nə tin var
Təkcə həvəsim yoxdu
Nə dərd var nə tin var
Xərcləməyə pulum var
Qulluğum var qulum var
Mən çıxanda mat qulum var
Xərcləməyə pulum var
Təkcə həvəsim yoxdu
Nə dərd var nə tin var
Xərcl�
